# 기본적인 세팅

1. gemma3:1b 허깅페이스에서 라이선스 동의를 받아놓아야 함

2. 허깅페이스 api key를 아래에 입력해야 함

3. corpus.json을 업로드 해놓아야 함

4. A100

In [1]:
!pip install trl huggingface_hub loguru -q
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from trl import DPOTrainer, DPOConfig
from loguru import logger
import shutil
import sys
import os
import itertools
import platform

logger.remove()
logger.add(
    sys.stdout,
    level="INFO",
    colorize=True,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <5}</level> | {message}"
)

# 버전 로깅
logger.info(f"python version       : {platform.python_version()}")
logger.info(f"torch version        : {torch.__version__}")
logger.info(f"transformers version : {__import__('transformers').__version__}")
logger.info(f"datasets version     : {__import__('datasets').__version__}")
logger.info(f"trl version          : {__import__('trl').__version__}")


# 데이터 로드
def load_raw_data(path="/content/corpus.json"):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# DPO용 전처리 함수
def make_dpo_data(raw_data):
    result = []
    for item in raw_data:
        instruction = item["instruction"]
        keywords    = item["input"]
        prompt      = f"{instruction}: {', '.join(keywords)}"
        chosen      = item["chosen"]
        rejected    = item["rejected"]

        result.append({
            "prompt":   prompt,
            "chosen":   chosen,
            "rejected": rejected
        })
    return result

def test(prompt, model, tokenizer, necessary_word):
    model = model.to("cuda")

    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 생성
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    # 디코딩
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    logger.info("="*100)
    logger.info(necessary_word + "\n" + prompt)
    logger.info(necessary_word + "\n" + generated_text)
    logger.info("="*100)
    return generated_text

13:23:10 | INFO  | python version       : 3.11.12
13:23:10 | INFO  | torch version        : 2.6.0+cu124
13:23:10 | INFO  | transformers version : 4.51.3
13:23:10 | INFO  | datasets version     : 3.6.0
13:23:10 | INFO  | trl version          : 0.17.0


In [3]:
# 하이퍼파라미터
num_epochs = 5
batch_size = 8
save_total_limit = 2
output_dir = "./outputs/dpo"
logging_dir = "./outputs/logs"
dpo_dir = "./outputs/best_dpo"

# 모델 및 토크나이저 설정
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
# 문학적 instruction 목록
instructions = [
    "제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요.",
    "아래 단어들을 활용해 상징과 감정이 담긴 문학적 단편을 작성해주세요.",
    "다음 키워드를 사용해 감성적이고 은유적인 이야기를 작성해주세요.",
    "아래 키워드를 사용해 비유와 상징이 녹아든 문학적 장면을 묘사해주세요."
]

# 키워드 목록
keywords_list = [
    ["밥", "숟가락", "그릇"],
    ["창문", "바람", "햇살"],
    ["우산", "비", "골목"],
    ["신발", "거리", "그림자"],
    ["책상", "연필", "종이"],
    ["시계", "벽", "침묵"],
    ["의자", "창가", "오후"],
    ["커피", "잔", "향기"],
    ["손", "온기", "기억"],
    ["길", "노을", "발자국"]
]

# 데카르트 곱을 이용해 모든 instruction-keywords 조합 생성
test_prompts = [
    f"{instruction} {', '.join(keywords)}"
    for instruction, keywords in itertools.product(instructions, keywords_list)
]

necessary_word = "[Before Train]"

before_response = []
for prompt in test_prompts:
    before_response.append(test(prompt, model, tokenizer, necessary_word))

13:23:38 | INFO  | ====================================================================================================
13:23:38 | INFO  | [Before Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇
13:23:38 | INFO  | [Before Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇, 식사, 식사장

1.  **밥을 먹고, 그릇을Filled으로 먹어 보는**Balcony에서 한 người의 집에서 밥을 먹고, 그릇을Filled으로 먹어 보는 것이 정상적인 식사 방식이라고 할 수 있습니다. 밥과 그릇은 식사와는 상호작용을 하는 수 있는 물리적 장소입니다.
2.  **수건을 쥐고, 식사장에 있는 사람들과 대화하는**어지러움을 줄이기 위해 수건을 쥐고, 식사장에 있는 사람들과 대화하는 것은 식사에 대한 감정과 기분을 높여주는 효과가 있습니다. 식사장에서 대화하는 사람들과는 식사 후의 관계를 형성할 수 있습니다.
3.  **다음과 함께 식사를 하다**다음과 함께 식사를 하다 means 다음의 음식을 먹을 수 있는 기회가 생기면 식사를 하다. 식사 후의 시간은 식사와 같은 감정과 기분을 형성하는 기회로, 다음과 함께 식사를 하다는 의미로 사용
13:23:38 | INFO  | ====================================================================================================
13:23:43 | INFO  | ====================================================================================================
13:23:43 | INFO  | [Before Train]
제공된 단어로 문학적인 어조로 짧은 장면을 

In [5]:
# pad_token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))
    logger.info(f"[Tokenizer] pad_token was None. Set to eos_token: {tokenizer.pad_token}")
else:
    logger.info(f"[Tokenizer] pad_token already set: {tokenizer.pad_token}")

# 데이터 전처리 및 분할
raw_data   = load_raw_data()
dpo_data   = make_dpo_data(raw_data)
dataset    = Dataset.from_list(dpo_data)
split      = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
eval_dataset  = split["test"]

logger.info(f"[Raw Data] First sample:\n{json.dumps(raw_data[0], ensure_ascii=False, indent=2)}")
logger.info(f"[DPO Records] First record:\n{json.dumps(dpo_data[0], ensure_ascii=False, indent=2)}")
logger.info(f"[Dataset] Total samples: {len(dataset)}")
logger.info(f"[Split] Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}")

13:27:06 | INFO  | [Tokenizer] pad_token was None. Set to eos_token: <|eot_id|>
13:27:06 | INFO  | [Raw Data] First sample:
{
  "instruction": "아래 세 단어를 사용해 감성적인 문학적 문장을 작성해주세요.",
  "input": [
    "김치",
    "냄새",
    "반찬"
  ],
  "rejected": "김치는 대표적인 발효 음식으로 한국 식탁에 빠지지 않는 반찬입니다. 특유의 냄새는 발효 과정에서 생성되는 유산균과 향신료에서 비롯됩니다. 김치는 보관 온도와 숙성 기간에 따라 맛이 달라집니다. 대부분의 한국 가정에서는 김치 냉장고를 통해 별도 보관합니다. 다양한 요리에 활용되며 건강에도 긍정적인 영향을 줍니다.",
  "chosen": "김치는 잊힌 계절처럼 식탁 위에 조용히 놓여 있었다. 냄새는 오래된 대화처럼 천천히 방 안을 채워나갔다. 반찬 중에서도 유독 진한 그 존재감은 입보다 기억을 먼저 자극했다. 그녀는 숟가락을 들기 전, 잠시 그 냄새에 고개를 숙였다. 어떤 향기는 맛보다 먼저 마음을 물들인다."
}
13:27:06 | INFO  | [DPO Records] First record:
{
  "prompt": "아래 세 단어를 사용해 감성적인 문학적 문장을 작성해주세요.: 김치, 냄새, 반찬",
  "chosen": "김치는 잊힌 계절처럼 식탁 위에 조용히 놓여 있었다. 냄새는 오래된 대화처럼 천천히 방 안을 채워나갔다. 반찬 중에서도 유독 진한 그 존재감은 입보다 기억을 먼저 자극했다. 그녀는 숟가락을 들기 전, 잠시 그 냄새에 고개를 숙였다. 어떤 향기는 맛보다 먼저 마음을 물들인다.",
  "rejected": "김치는 대표적인 발효 음식으로 한국 식탁에 빠지지 않는 반찬입니다. 특유의 냄새는 발효 과정에서 생성되는 유산균과 향신료에서 비롯됩니다. 김치는 보관 온도와 숙성 기간에 따라 맛이 달라집니다. 대부분의 한국 가

In [6]:
# 총 step 계산
total_train_steps = len(train_dataset) // batch_size * num_epochs
logging_steps     = max(1, total_train_steps // (num_epochs * 2))
save_steps        = logging_steps

logger.info(f"Total samples (train): {len(train_dataset)}")
logger.info(f"Total samples (eval): {len(eval_dataset)}")
logger.info(f"Batch size: {batch_size}")
logger.info(f"Epochs: {num_epochs}")
logger.info(f"Total training steps: {total_train_steps}")
logger.info(f"Logging steps: {logging_steps}")
logger.info(f"Save steps: {save_steps}")

13:27:06 | INFO  | Total samples (train): 140
13:27:06 | INFO  | Total samples (eval): 35
13:27:06 | INFO  | Batch size: 8
13:27:06 | INFO  | Epochs: 5
13:27:06 | INFO  | Total training steps: 85
13:27:06 | INFO  | Logging steps: 8
13:27:06 | INFO  | Save steps: 8


In [7]:
# DPOConfig
training_args = DPOConfig(
    beta=0.1,
    max_length=512,
    max_prompt_length=192,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_dir=logging_dir,
    logging_steps=logging_steps,
    save_strategy="steps",
    save_steps=save_steps,
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=logging_steps,
    save_total_limit=save_total_limit,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    fp16=True
)

# Trainer 정의
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer
)

# 학습 실행
trainer.train()

# 모델 저장
trainer.model.save_pretrained(dpo_dir)
tokenizer.save_pretrained(dpo_dir)
shutil.make_archive("best_dpo", 'zip', dpo_dir)

Extracting prompt in train dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/140 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/35 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
8,0.562100,0.121559,1.227637,-0.986385,1.000000,2.214022,-417.196198,-396.594330,-1.015436,-0.875663
16,0.038000,0.001084,2.248026,-5.733219,1.000000,7.981245,-406.992279,-444.062683,-1.160583,-1.165985
24,0.000200,0.000030,2.163878,-10.380838,1.000000,12.544716,-407.833771,-490.538879,-1.207481,-1.292253
32,0.000000,0.000008,2.002589,-12.319868,1.000000,14.322456,-409.446625,-509.929138,-1.224154,-1.333436
40,0.000000,0.000005,1.921572,-13.120878,1.000000,15.042452,-410.256775,-517.939270,-1.230693,-1.349110
48,0.000000,0.000004,1.882507,-13.453891,1.000000,15.336398,-410.647461,-521.269409,-1.233386,-1.355344
56,0.000000,0.000004,1.867375,-13.586777,1.000000,15.454153,-410.798767,-522.598267,-1.234443,-1.357908
64,0.000000,0.000004,1.861678,-13.640322,1.000000,15.501999,-410.855774,-523.133728,-1.234674,-1.358747
72,0.000000,0.000003,1.859690,-13.659352,1.000000,15.519043,-410.875641,-523.324036,-1.234813,-1.359105
80,0.000000,0.000003,1.858384,-13.663275,1.000000,15.521660,-410.888733,-523.363220,-1.234937,-1.359361


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


'/content/best_dpo.zip'

In [8]:
total_size = sum(os.path.getsize(os.path.join(root, f))
                 for root, _, files in os.walk(dpo_dir)
                 for f in files)

logger.info(f"Total size of best_dpo: {total_size / (1024**2):.2f} MB")

13:52:34 | INFO  | Total size of best_dpo: 4730.74 MB


In [9]:
test_prompts = [
    f"{instruction} {', '.join(keywords)}"
    for instruction, keywords in itertools.product(instructions, keywords_list)
]

necessary_word = "[After Train]"

after_response = []
for prompt in test_prompts:
    after_response.append(test(prompt, model, tokenizer, necessary_word))

13:52:41 | INFO  | ====================================================================================================
13:52:41 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇
13:52:41 | INFO  | [After Train]
제공된 단어로 문학적인 어조로 짧은 장면을 창작해주세요. 밥, 숟가락, 그릇, 한숨... 두려운 silence를 향해 고요한 곳에서 자서를 말해주었다.

그날, 밥과 숟가락이 그의 발목을 물리치고, 그릇이 그의 아마도 한 번만 한 번만 말할 것이라고 생각했다. 그는 어ígged을 때, 숟가락은 그의 몸에 쌓여 있었고 그릇은 그곳에서 잠시 뻗어내렸다. 그는 한숨을 내었다. 그때, 그는 그곳에서 어쨌든 말하던 silence는, 어쨌든 말하지 않는 silence였다.

그는 한 번 더 그 place에서 살았던 그 silence를 가로지르기 위해, 그 place에서 살고 있었던 silence를 가로지르기 위해. 그는 그 place에서 살고 있었던 silence를 그 place에서 살고 있었던 silence의 모음처럼, 한 번만 한 번만 말할 것이라고 생각했다. 그는 한숨을 내었다. 그때, 그는 그 place에서 살고 있었던 silence를 가로지르기 위해, 그 place에서 살고 있었던 silence의 모음처럼, 한 번만
13:52:41 | INFO  | ====================================================================================================
13:52:47 | INFO  | ====================================================================================================
13:52:47 | I